# PSTAT 100 Project report

This document provides a template that you can elect to fill in or modify for your project report. The sectioning (header structure) is not strict, and you are encouraged to adjust it to suit your project. **If you choose to use this template, please remove all guiding text, including this cell**.

#### General guidelines

The objective of your final report is to provide a thorough overview of your project. It need not be long; quality and clarity is preferred over quantity. Aim for 3-5 pages; if you feel that further material needs to be included, you can add an appendix with supplementary information, tables, and plots.

#### Contents

The report should include the following elements:

* Abstract
    + One-paragraph summary of the report contents.
* Background
    + Adapt your interim report.
* Aims
    + State the specific questions and approaches you took up.
* Data description
    + Adapt your interim report.
    + Possibly include select results of your exploratory analysis.
* Methods 
    + Description of the methods used in your analysis.
* Results
    + Present the figures and tables summarizing your analysis.
* Discussion
    + Highlight your main findings and takeaways.
    + Offer further commentary: caveats, further steps, etc.

These need not be set apart by headers; you are free to determine how to organize your report. For an example, please refer to the adaptation of HW3 into a report provided during week 8.

#### Format and appearance 

* No codes should appear in your report.
* All figures and tables should have captions.
* Figures should be appropriately sized and labeled.
* No text from the template should appear in your report other than headers.
* The total length should not exceed 8 pages.

### Evaluation

Your report will be evaluated based on:
* (format) adherence to formatting and appearance guidelines;
* (clarity) clarity and thoughtfulness in written voice;
* (accuracy) apparent accuracy of quantitative results and technical information;
* (applied a PSTAT100 technique) successful use of one or more techniques in the course.

Notice that no credit is tied to the nature of the results; you can earn credit equally well with an analysis that says little as with one that says a lot. **Negative, neutral, or ambiguous results -- analyses that do not produce any particular insight -- are more than acceptable.** If your analyses turn in one of these directions, present them as clearly as possible, and consider speculating in your discussion section about the absence of signficicant/interpretable findings.

---

# Project title

Author 1, Author 2, Author 3, and Author 4.

#### Author contributions

Author 1 contributed ...

Author 2 contributed ...

#### Abstract

Prepare an abstract *after* you've written the entire report. The abstract should be 4-6 sentences summarizing the report contents. Typically:
* the first 1-2 sentences introduce and motivate the topic;
* the next 1-2 sentences state the aims;
* the next 1-2 sentences state the findings.

---
## Introduction

The goal of this section is to introduce your project topic and project aims (the questions you posed in the planning report).

### Background

Provide enough background to position the reader to understand your project aims and their relevance. Most likely, you can adapt your background section from the plan report for this section. If you were satisfied with what you wrote previously and no revisions were suggested, you can simply copy that material. 

You may want to consider making revisions anyway now that you've carried out your data analysis -- perhaps some pieces of background information aren't as relevant to understanding your work, or some pieces need to be elaborated in further detail.

### Aims

Indicate your project aims, approaches, and findings in a paragraph or two. This should be non-technical, but detailed enough to enable the reader to imagine the contents of the report that follows. Most likely, you can adapt your 'planned work' section from the planning report, stating each aim, appraoch, and the primary finding in turn. If your analysis produced null or negative results, you can say as much here.

In [124]:
import numpy as np
import pandas as pd
import altair as alt
import sklearn.linear_model as lm
import warnings
from sklearn.preprocessing import add_dummy_feature
from vega_datasets import data
warnings.simplefilter(action='ignore', category=FutureWarning)

In [125]:
pwd

'/Users/davidhong/Desktop/pstat100/Class Project/CP2'

In [126]:
dataset=pd.read_csv('data/tidy-data copy.csv')
dataset=dataset.drop(columns=['Unnamed: 0'])
dataset_every_day = dataset.loc[dataset['Response'] == 'Every Day']
dataset_every_day=dataset_every_day[dataset_every_day['Gender']!='Overall']
dataset_every_day=dataset_every_day[dataset_every_day['Data Value']!='*']
legislation_by_state = dataset_every_day.groupby('Location Description')['legislation percentage'].mean().sort_values().reset_index()

alt.Chart(dataset_every_day).mark_bar().encode(
    x=alt.X('Location Description:N', sort='-y'),
    y=alt.Y('Data Value:Q'),
    tooltip=['Location Description', 'Data Value']
).properties(
    width=500,
    height=300,
    title='Smoking Percentage by State'
)

alt.Chart(...)

In [127]:
alt.Chart(dataset_every_day).mark_circle().encode(
    alt.X('legislation percentage:Q',title='Legislation Percentage',
          scale=alt.Scale(domain=[70, 100])),
    alt.Y('Data Value:Q', title='Smoking Prevalence'),
    alt.Color('Topic Description:N'),
    tooltip=['Location Description', 'legislation percentage', 
             'Data Value', 'Topic Description']
).properties(
    width=400,
    height=300,
    title='Scatter plot of Legislation Percentage vs. Data Value (Every Day)'
)

alt.Chart(...)

In [128]:
alt.Chart(dataset_every_day).mark_circle().encode(
    alt.X('legislation percentage:Q',title='Legislation Percentage',
          scale=alt.Scale(domain=[70, 100])),
    alt.Y('Data Value:Q', title='Smoking Prevalence'),
    alt.Color('Gender:N')
).properties(
    width=400,
    height=300,
    title='Scatter plot of Legislation Percentage vs. Data Value (Every Day)'
)

alt.Chart(...)

In [129]:
x_dummy=dataset_every_day.drop(columns=['Year','Data Value','Low Confidence Limit','High Confidence Limit','Sample Size','legislation percentage'
                                        ,'Measure Description','Location Description','Response'])
x_df=pd.get_dummies(x_dummy,drop_first=True)
x_df['legislation']=dataset_every_day['legislation percentage']
x_mx=add_dummy_feature(x_df, value = 1)

In [130]:
y=dataset_every_day['Data Value']
y=pd.to_numeric(y) #y is stored as string in the dataframe
y=y.to_numpy()


In [131]:
mlr = lm.LinearRegression(fit_intercept = False)
mlr.fit(x_mx,y)

LinearRegression(fit_intercept=False)

In [132]:
# store dimensions
n=x_mx.shape[0]
p=x_mx.shape[1]
# compute x'x
xtx = x_mx.transpose().dot(x_mx)
          # compute x'x inverse
xtx_inv = np.linalg.inv(xtx)
          # compute residuals
resid = y-mlr.predict(x_mx)
          # compute error variance estimate
sigmasqhat = ((n-1)/(n-p)) * resid.var()
          # compute variance-covariance matrix
v_hat = xtx_inv * sigmasqhat
          # compute standard errors
coef_se = np.sqrt(v_hat.diagonal())
coef_se = np.append(coef_se,float('nan'))
          # coefficient labels
newarray=x_df.columns.insert(0,'intercept')
coef_labels=list(newarray)
coef_labels.append('error_variance')
# estimates
coef_estimates= np.append(mlr.coef_,sigmasqhat)
          # summary table
coef_table = pd.DataFrame(data={'coef_estimates':coef_estimates,'coef_se':coef_se},index=coef_labels)
coef_table

,coef_estimates,coef_se
intercept,102.758916,4.731984
Topic Description_E-Cigarette Use (Adults),-38.190083,0.627385
Gender_Male,-0.379051,0.495846
legislation,-0.379752,0.055362
error_variance,31.101691,NaN


In [133]:

visuals=x_df
for i in range(0,len(visuals)):
    if visuals.iloc[i,0] ==1:
        visuals.iloc[i,0]='e-cigar'
    elif visuals.iloc[i,0] ==0:
        visuals.iloc[i,0]='cigar'
for i in range(0,len(visuals)):
    if visuals.iloc[i,1] ==1:
        visuals.iloc[i,1]='Male'
    elif visuals.iloc[i,1] ==0:
        visuals.iloc[i,1]='Female'
visuals=visuals.reset_index()
visuals=visuals.drop(columns=['index'])
visuals['predicted values']=mlr. predict(x_mx)
visuals['actual values']=y
visuals['residuals'] = visuals['actual values'] - visuals['predicted values']
scatter_chart = alt.Chart(visuals).mark_circle().encode(
    x=alt.X('legislation', axis=alt.Axis(title='legislation percentage'),
           scale=alt.Scale(domain=[70, 100])),
    y=alt.Y('actual values', axis=alt.Axis(title='Actual smoking percentage')),
    color=alt.Color('Topic Description_E-Cigarette Use (Adults)')
)
# construct Regression scatter plot + line
mlr_lines = scatter_chart.mark_line(color = 'red').encode(y = 'predicted values')
scatter_chart + mlr_lines

alt.LayerChart(...)

In [134]:
visuals.head()


,Topic Description_E-Cigarette Use (Adults),Gender_Male,legislation,predicted values,actual values,residuals
0,cigar,Male,80.2,71.923742,67.1,-4.823742
1,cigar,Female,80.2,72.302794,70.0,-2.302794
2,cigar,Male,80.2,71.923742,70.3,-1.623742
3,cigar,Female,80.2,72.302794,72.7,0.397206
4,cigar,Male,80.2,71.923742,71.0,-0.923742


In [135]:
lines1 = alt.Chart(visuals).mark_circle().encode(
x=alt. X('predicted values',axis= alt. Axis(title= 'predicted values')), 
y=alt.Y('residuals',axis=alt.Axis(title='Residuals on the estimate')),
color= alt. Color('Topic Description_E-Cigarette Use (Adults)')
)
lines1

alt.Chart(...)

## Map analysis part code

We can create a choropleth map of the United States, where each state is shaded according to the smoking prevalence, with the shade of color varying based on the level of legislation percentage in that state. This could help visualize any potential patterns or trends in smoking rates and legislation percentage across different states. Additionally, a regression analysis could be conducted to see if there is a significant relationship between smoking rates and legislation percentage, while controlling for potential confounding variables such as age and gender.

In [136]:
# Filter the dataset to the year you're interested in
year = 2015  # Replace with the desired year
filtered_data = dataset_every_day[dataset_every_day['Year'] == year]
ansi = pd.read_csv('https://www2.census.gov/geo/docs/reference/state.txt', sep='|')
ansi.columns = ['id', 'abbr', 'state', 'statens']
ansi = ansi[['id', 'abbr', 'state']]

filtered_data1 = pd.merge(filtered_data, ansi, how='left',
                         left_on='Location Description', right_on='state')

# Load the United States map data
states = alt.topo_feature(data.us_10m.url, feature='states')

# Create the choropleth map of legislation
base = alt.Chart(states).mark_geoshape(fill='lightgray', stroke='black', strokeWidth=0.5)

chart = alt.Chart(states).mark_geoshape(stroke='black').encode(
    alt.Color('legislation percentage:Q', legend=alt.Legend(title="Legislation Percentage")),
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(filtered_data1, 'id', ['legislation percentage'])
).properties(
    width=400,
    height=300,
    title='Legislation Percentage by State'
).project('albersUsa')
chart1 = base + chart
# Create the choropleth map of legislation
base = alt.Chart(states).mark_geoshape(fill='lightgray', stroke='black', strokeWidth=0.5)

chart = alt.Chart(states).mark_geoshape(stroke='black').encode(
    alt.Color('Data Value:Q', legend=alt.Legend(title="Smoking Prevalence")),
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(filtered_data1, 'id', ['Data Value'])
).properties(
    width=400,
    height=300,
    title='Smoking Prevalence by State'
).project('albersUsa')
chart2 = base + chart
alt.vconcat(chart1, chart2).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)

alt.VConcatChart(...)

In [137]:
for i in range(0,506):
    if visuals.iloc[i,0]=='cigar':
        visuals.iloc[i,0]=1
    else:
        visuals.iloc[i,0]=0
for i in range(0,506):
    if visuals.iloc[i,1]=='Male':
        visuals.iloc[i,1]=1
    else:
        visuals.iloc[i,1]=0
visuals=visuals.drop(columns=['actual values','residuals'])
visuals.head()

,Topic Description_E-Cigarette Use (Adults),Gender_Male,legislation,predicted values
0,1,1,80.2,71.923742
1,1,0,80.2,72.302794
2,1,1,80.2,71.923742
3,1,0,80.2,72.302794
4,1,1,80.2,71.923742


In [144]:
a=visuals.to_numpy()
a=a.astype(float)
corrmatrix=np.corrcoef(a.T)
corrtable=pd.DataFrame(corrmatrix)
corrtable=corrtable.rename(columns={0:"if it is cigar", 1:"if it is male",2:"legislation rate",3:"predicted values" })
corrtable=corrtable.rename(index={0:"if it is cigar", 1:"if it is male",2:"legislation rate",3:"predicted values" })
corrtable

,if it is cigar,if it is male,legislation rate,predicted values
if it is cigar,1.000000,0.000000,0.006808,0.993624
if it is male,0.000000,1.000000,0.000000,-0.012488
legislation rate,0.006808,0.000000,1.000000,-0.105284
predicted values,0.993624,-0.012488,-0.105284,1.000000


In [146]:
corr_mx_long = corrtable.reset_index().rename(
columns = {'index': 'row'}
).melt(
id_vars = 'row',
var_name = 'col',
value_name = 'Correlation')
alt.Chart(corr_mx_long).mark_rect().encode(
)
# construct plot
alt.Chart(corr_mx_long).mark_rect().encode(
    x = alt.X('col', title = '', sort = {'field': 'Correlation', 'order': 'ascending'}), 
    y = alt.Y('row', title = '', sort = {'field': 'Correlation', 'order': 'ascending'}),
    color = alt.Color('Correlation', 
                      scale = alt.Scale(scheme = 'blueorange', # diverging gradient
                                        domain = (-1, 1), # ensure white = 0
                                        type = 'sqrt'), # adjust gradient scale
                     legend = alt.Legend(tickCount = 5)) # add ticks to colorbar at 0.5 for reference
).properties(width = 300, height = 300)


alt.Chart(...)

---
## Materials and methods

The goal of this section is to describe your dataset(s) and sketch out your analysis.

### Datasets

For this section, adapt the data description from your planning report. All the same information should appear, but in contrast to the planning report, it should have narrative structure (read smoothly) and **not** contain sub-headers (basic information, data semantics and structure).

Most likely, you will want to include two tables in this section: the variable descriptions you prepared for the planning report; and the example rows you showed in your planning report.

### Methods

In order to explore the relationship between smoking percentage and other predictors, I ran a regression analysis over the dataset. I dropped several variables because they do no fit in the regression model, such as the state name, upper and lower confidence interval. I set the genders and cigar/ecigar to be dummy variables and the legislation rate to be the quantitative variables. The results show that the gender coefficient is not significant with only -0.37. Thus, it is safe to say we can ignore that coefficient. I then plotted the regression lines with two different intercepts.   

---
## Results

### Regression results
the results show that people have less tendency to smoke ecigar and legislation rate does affect the smoking percentage in a negative way. It also shows that the eciagr's predicted values have larger variance where as the cigars' predicted values are more condensend(less variance). 
### Map results
On the choreograph, we can see that legislation rate is more significant on the west side of united states, and the smoking percentage is relatively higher than the east side of the country. Moreover, we can see that two choreographs have direct opposite distributions of colors, meaning that the legislation rate and smoking percentage does have negative relationship